### Imports

In [5]:
import matplotlib.pyplot as plt
from matplotlib import cm, colors
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from scipy.special import sph_harm
from matplotlib import animation
from matplotlib.colors import LightSource
from IPython import display
import matplotlib

from dataloader import loadhcp
from scipy.special import sph_harm

# Spherical harmonics

Spherical harmonics are special functions defined on the surface of a sphere. "Since the spherical harmonics form a complete set of orthogonal functions and thus an orthonormal basis, each function defined on the surface of a sphere can be written as a sum of these spherical harmonics." Spherical harmonics are the extension of Fourier series to higher dimensions. As sines and cosines in Fourier series forms an orthonormal basis for functions on the circle (S(1) group), spherical harmonics form an orthonormal basis for any spherical function (S(2) group)i.e. any function defined on the surface of a sphere can be written as a sum of spherical harmonics. Thus spherical harmonics provide a smooth representation of data distributed on a sphere. Similarly to the sines and cosines in Fourier series, the spherical harmonics can also be ordered by angular frequency (grouped in rows):

![SPHERICAL_HARMONICS_VISUALIZATION](./figures/spherical_harmonics_visualization.png)

"Further, spherical harmonics are basis functions for irreducible representations of SO(3), the group of rotations in three dimensions."

"Spherical harmonics emerge from solving Laplace's equation in a spherical domain."

Spherical harmonics of degree $l$ and order $m$, where $l\geq 0$ and $-l\leq m\leq l$ is defined as:

$$Y^{m}_{l}(\theta ,\phi )=\sqrt{\frac{(2l+1)}{4\pi }\frac{(l-m)!}{(l+m)!}}P^{m}_{l}(cos \, \theta)e^{im\phi }$$

Where $P^{m}_{l}$ is the associated Legendre polynomial defined as:

$$P^{m}_{l}(x)=(-1)^{m}(1-x^{2})^{\frac{m}{2}}\frac{d^{m}}{dx^{m}}P_{l}(x)$$

where

$$P_{l}(x)=\sum_{k=0}^{\infty}\frac{(-v)_{k}(v+1)_{k}}{(k!)^{2}}\left ( \frac{1-x}{2} \right )^{k}$$

and $(\cdot)_{k}$ is the Pochhammer symbol defined as:

$$(x)_{k}=\frac{\Gamma(x+k)}{\Gamma(x)}$$

The order $m$ corresponds to the angular frequency of the basis function i.e. how many full oscilations do we encounter when going all the way around some equator (some constant latitude) on the sphere. And the degree $l$ corresponds to different orthogonal modes at given angular frequency $m$. For a given value of $l$, there are $2l + 1$ independent solutions of this form, one for each integer $m$.

The colatitude $\theta$, or polar angle, ranges from $0$ at the North Pole, to $\frac{\pi}{2}$ at the Equator, to $\pi$ at the South Pole, and the longitude $\phi$, or azimuth, may assume all values with $0 \leq \phi < 2\pi$.

In a spherical coordinate system, a colatitude is the complementary angle of a given latitude, i.e. the difference between a right angle and the latitude.[1] Here Southern latitudes are defined to be negative, and as a result the colatitude is a non-negative quantity, ranging from zero at the North pole to 180° at the South pole.

# Spherical harmonics visualization

In [2]:
from visualization.spherical_harmonics_visualization import visualizeAllHarmonicsOfDegree

visualizeAllHarmonicsOfDegree(1,resolution=200)

1 -1
Drawing ...


1 0


C:\Users\ressl\Desktop\MSc Project\AlesMScProject\visualization\spherical_harmonics_visualization.py:21: RuntimeWarning: invalid value encountered in divide
  fcolors = (fcolors - fmin)/(fmax - fmin)


Drawing ...


1 1
Drawing ...


# Spherical harmonics expansion

Any well-behaved function $f(\theta,\phi)$ on the sphere can be expanded to spherical harmonics basis as follows:

$$f(\theta, \phi) = \sum_{l=0}^{\infty }\sum_{m=-l}^{l}c^{m}_{l}Y^{m}_{l}(\theta, \phi)$$

where $c^{m}_{l}$ is the expansion coefficient for some spherical harmonic $Y^{m}_{l}(\theta, \phi)$ defined as:

$$c^{m}_{l} = \int_0^{2\pi} d\phi  \int_0^{\pi} d\theta sin(\theta) Y_{l}^{m*} (\theta,\phi) f(\theta,\phi)$$

Smooth functions with negligible high angular frequency content can be approximated with little to no loss by using harmonics only up to degree $l_{max}$:

$$f(\theta, \phi) = \sum_{l=0}^{l_{max}}\sum_{m=-l}^{l}c^{m}_{l}Y^{m}_{l}(\theta, \phi)$$

Since the dMRI signals are real-valued and antipodlly symmetric (i.e. symmetric about the origin $f(x)=f(-x)$) simplified spherical harmonic basis can be used. The basis is real thus without imaginary components. Furthermore due to the atipodal symmetry all spherical harmonics with odd degree can be set to $0$. This is because the diffusion is symmetric about the origin e.g. diffusion in the direction $[1,1,1]$ is the same as diffusion in the direction $[-1,-1,-1]$. And since spherical harmonics have the following parity property:

$$Y^{m}_{l}(\theta,\phi)=(-1)^{m}\overline{Y^{-m}_{l}(\theta,\phi)}$$

and also

$$Y^{m}_{l}(\pi-\theta,\pi+\phi)=(-1)^{l}Y^{m}_{l}(\theta,\phi)$$

it is now possible to see that only spherical hermonics with even degrees correspond to antipodal terms (the other spherical harmonics that are not symmetric around the origin are not needed in our case). Thus the resulting simplified basis becomes:

$$\begin{split}Y_{lm}(\theta,\phi) = \begin{cases}
0 & \text{if $l$ is odd}, \\
\sqrt{2} \: \text{Im} \left[ Y_l^{-m}(\theta,\phi) \right] & \text{if $m < 0$},\\
Y_l^0(\theta,\phi) & \text{if $m = 0$},\\
\sqrt{2} \: \text{Re} \left[ Y_l^m(\theta,\phi) \right] & \text{if $m > 0$},\\
\end{cases}\end{split}$$


------------NOTE------------

The real spherical harmonics $Y_{lm}$ with $m > 0$ are said to be of cosine type, and those with $m < 0$ of sine type. The main difference between sine-type and cosine-type spherical harmonics is the way they depend on the azimuthal angle. Cosine-type spherical harmonics are even functions with respect to $\phi$, while sine-type spherical harmonics are odd functions with respect to $\phi$.

----------------------------

# Least-squares expansion coefficient approximation

Even though the theoretical results above are valid, the coefficients of the spherical harmonics expansion need to be computed only from samples of $f(\theta,\phi)$ since the function is unknown. One possibility is to use Least-Squares approximation to compute the spherical harmonics expansion. However this method makes assumptions about the sampling grid. On the other hand the Gram-Schmidt orthogonalization does not make these assumpotions and guarentees that the inverse transform converges to the sample points. Thus the spherical convolution can be performed even on spheres without the McEwen-Wiaux, Driscoll-Healy or HEALPix sampling schemes. See the sampling schemes below:

![SPHERE_SAMPLING_SCHEMES](./figures/sphere_sampling_schemes.png)

Let spherical functions ($L^{2}(S^{2})$) be the Hilbert space of square integrable functions on the 2-dimensional sphere $S^{2}$ and the coordinates are defined as colatitude $\theta$ and longitude $\phi$. Square integrable functions need to satisfy this condition:

$$\int_{-\infty }^{\infty }\left | f(x) \right |^{2}dx < \infty$$

where $f(x)$ is a real- or complex-valued function.

The inner product on the sphere is defined as:

$$
\left \langle f,h \right \rangle=\int_{0}^{\pi}\int_{0}^{2\pi}f(\theta,\phi)\overline{h(\theta,\phi)}\, sin\theta \, d\phi d\theta
$$

Thus the expansion coefficients can also be defined as:

$$\widehat{f}(l,m)=\left \langle f,Y^{m}_{l} \right \rangle=c^{m}_{l}$$

Since it is not possible to discretize the surface of the sphere in a way that the neigbourhood of each point is the same therefore a common approach to performing a spherical convolution is to express the discretized spherical function and a filter in the spherical harmonics domain and perform the convolution there. This approach is supported by the Sampling Theorem, Convolution Theorem and fast spherical transform. The Sampling Theorem guarantees the reversibility of the discretization of the spherical functions. And the fast spherical transform ensures computational efficiency.

## The Convolution Theorem
The Convolution Theorem states that expansion coefficient of a convolution of some spherical functions $f$ and $h$ is a product of the expansion coefficients of $f$ and $h$:

$$(\widehat{f*h})(l,m)=2\pi\sqrt{\frac{4\pi}{2l+1}}\widehat{f}(l,m)\widehat{h}(l,0)$$

Since this theorem is independent of sampling it is therefore possible to perform convolution in the spherical harmonics domain as long as the projection of the functions onto the sperical harmonics domain is accurate. Due to this it is possible to transform the spherical functions $f$ and $h$ into the spherical harmonics domain, perform convolution there and the perform the inverse transform to get the convolution $f*h$.

## The Sampling Theorem
The Sampling Theorem states that for a bandlimited ($\widehat{f}(l,m)=0$ for $l\geq b$) spherical function $f$ of bandwidth $b$ the expansion coefficients can be calculated as:

$$\widehat{f}(l,m)=\frac{\sqrt{2\pi}}{2b}\sum_{j=0}^{2b-1}\sum_{k=0}^{2b-1}a^{(j)}_{b}f(\theta_{j},\phi_{k})\overline{Y^{l}_{m}(\theta_{j},\phi_{k})}$$

where $l<b$, $\left | m \right |\leq l$, $\theta_{j}=\pi\frac{2j+1}{4B}$, $\theta_{k}=\frac{\pi k}{b}$ and $a^{(b)}_{j}$ are weights that compensate for the oversampling at the poles. It is important to note that according to the theorem $(2b)^{2}$ samples are sufficient to be able to reconstruct $f$. However the assumption of this theorem is that the sphere is sampled as a lat-lon grid.

## The Uniform Resolution Theorem
The Uniform Resolution Theorem states that each spherical harmonic of some degree $l$ can be rotated and then expressed as a linear combination of only the harmonics of the same degree $l$. Therefore the impossibility of sphere discretization can be circumvented.

## Additional properties of real spherical functions
Since the MRI data is real-valued additional properties of real-valued spherical harmonics can be derived:

\begin{align}
c^{-m}_{l} &= \int_{0}^{\pi}\int_{0}^{2\pi}f(\theta,\phi)\overline{Y^{-m}_{l}(\theta,\phi)}sin\theta\, d\phi\, d\theta \\
& = \int_{0}^{\pi}\int_{0}^{2\pi}f(\theta,\phi)[(-1)^{m}Y^{m}_{l}(\theta,\phi)]sin\theta\, d\phi\, d\theta \\
& = (-1)^{m}\overline{\int_{0}^{\pi}\int_{0}^{2\pi}f(\theta,\phi)\overline{Y^{m}_{l}(\theta,\phi)}sin\theta\, d\phi\, d\theta} \\
& = (-1)^{m}\overline{c^{m}_{l}}
\end{align}

Thus we can merge expansion coefficients $c^{-m}_{l}$ and $c^{m}_{l}$ into a single coefficient $C^{m}_{l}$ with $m>0$ as follows:

\begin{align}
C^{m}_{l} &= c^{-m}_{l}Y^{-m}_{l} + c^{m}_{l}Y^{m}_{l} \\
& = (-1)^{m}\overline{c^{m}_{l}}(-1)^{m}\overline{Y^{m}_{l}}+ c^{m}_{l}Y^{m}_{l}\\
& = \overline{c^{m}_{l}Y^{m}_{l}} + c^{m}_{l}Y^{m}_{l}\\
& = 2Re(c^{m}_{l}Y^{m}_{l}) \\
& = 2[a^{m}_{l}R^{m}_{l}+b^{m}_{l}(-I^{m}_{l})]
\end{align}

where $R^{m}_{l}$ is the real part of $Y^{m}_{l}$ and $I^{m}_{l}$ is the imaginary part of $Y^{m}_{l}$. Note that the derivation above holds only for $m$ > 0 since when $m=0$ there is no other spherical harmonic with $-m$ since $-0=0$ therefore the same spherical harmonic would be included twice. Thus $C^{0}_{l}=c^{0}_{l}$ and since $Y^{m}_{l}$ is real therefore $c^{0}_{l}$ must also be real since $f$ is real.

------------NOTE------------

Hilbert space: https://www.youtube.com/watch?v=_kJUUxjJ_FY

----------------------------

## Computation

In [6]:
#Load data
bvals, qhat, dwis = loadhcp.load_hcp()

dwis.shape

(108, 145, 174, 145)

In [ ]:
# Epand data to full sphere
temp = np.zeros((3,len(dwis)*2))
for i in range(len(qhat)):
    temp[i] = np.concatenate((qhat[i], -qhat[i]), axis=0)
    
qhat=temp

extended_shape = (len(dwis)*2,145,174,145)

temp = np.zeros(extended_shape)
for i in range(dwis.shape[1]):
    for j in range(dwis.shape[2]):
        for k in range(dwis.shape[3]):
            temp[:,i,j,k] = np.concatenate((dwis[:,i,j,k], dwis[:,i,j,k]), axis=0)

dwis=temp

bvals = np.concatenate((bvals, bvals), axis=0)
bvals.shape

In [7]:
# Visualize the data
%matplotlib qt

voxel = dwis[:,100,75,71]

fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")
ax.scatter(qhat[0],qhat[1],qhat[2],c=voxel,vmin=1000,vmax=2000)
plt.show()

ImportError: Cannot load backend 'Qt5Agg' which requires the 'qt' interactive framework, as 'tk' is currently running

In [ ]:
# Transform Cartesian coordinates to spherical

thetas = np.arccos(qhat[2])
phis = np.arctan2(qhat[1], qhat[0]) + np.pi / 2

In [9]:
def spherical_harmonic_basis(l, m, thetas, phis):
    if l % 2 == 1:
        return np.zeros(len(thetas))
    if m < 0:
        return np.sqrt(2) * sph_harm(-m, l, phis, thetas).imag
    if m == 0:
        return sph_harm(m, l, phis, thetas).real
    if m > 0:
        return np.sqrt(2) * sph_harm(m, l, phis, thetas).real

In [10]:
max_degree = 6
number_of_coefficients = int(0.5 * (max_degree + 1) * (max_degree + 2))

def get_storage_index(l,m):
    return int(0.5 * l * (l + 1) + m)

B = np.zeros((dwis.shape[0], number_of_coefficients))

for l in range(0, max_degree + 1, 2):
    for m in range(-l, l + 1):
        B[:, get_storage_index(l,m)] = spherical_harmonic_basis(l, m, thetas, phis)
        
B

NameError: name 'thetas' is not defined

In [ ]:
expansion_coefficients = np.linalg.pinv(B.T @ B) @ B.T @ voxel
expansion_coefficients.shape

### Signal reconstruction

In [ ]:
original_signal = dwis[1,100,75,71]
theta = thetas[1]
phi = phis[1]

cummulative_sum = 0
for l in range(0, max_degree + 1, 2):
    for m in range(-l, l + 1):
        storage_index = get_storage_index(l,m)
        
        cummulative_sum += expansion_coefficients[storage_index]*spherical_harmonic_basis(l,m,np.array([theta]),np.array([phi]))[0]
        
reconstructed_signal = cummulative_sum

print(original_signal)
print(reconstructed_signal)